In [2]:
from itertools import combinations


generate 35 permutations of 4 groups - 8 choose 4, but will some repeats where incl = x, excl = y vs incl = y, excl = x

In [ ]:
# how to actually do this -- 
best_concepts = [
    "A.Fayed", 
    "R.Wallace", 
    "T.Lennox", 
    "N.Yassir", 
    "K.Hayes", 
    "M.OBrian", 
    "J.Bauer", 
    "C.Manning"
]

permutations = list(combinations(best_concepts, 4))

for i, p in enumerate(permutations[:5]):
    complement = set([c for c in best_concepts if c not in p])
    permutations[i] = [set(p), complement]

unique = [] # keep all whose thing is first? how to do
for 


[[{'A.Fayed', 'N.Yassir', 'R.Wallace', 'T.Lennox'},
  {'C.Manning', 'J.Bauer', 'K.Hayes', 'M.OBrian'}],
 [{'A.Fayed', 'K.Hayes', 'R.Wallace', 'T.Lennox'},
  {'C.Manning', 'J.Bauer', 'M.OBrian', 'N.Yassir'}],
 [{'A.Fayed', 'M.OBrian', 'R.Wallace', 'T.Lennox'},
  {'C.Manning', 'J.Bauer', 'K.Hayes', 'N.Yassir'}],
 [{'A.Fayed', 'J.Bauer', 'R.Wallace', 'T.Lennox'},
  {'C.Manning', 'K.Hayes', 'M.OBrian', 'N.Yassir'}],
 [{'A.Fayed', 'C.Manning', 'R.Wallace', 'T.Lennox'},
  {'J.Bauer', 'K.Hayes', 'M.OBrian', 'N.Yassir'}]]